In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd '/content/drive/MyDrive/'
import pandas as pd
from sklearn.metrics import r2_score
import re
import math
from nltk.stem.snowball import SnowballStemmer
import nltk
import  matplotlib.pyplot as plt
from nltk.corpus import stopwords
nltk.download('stopwords')
import seaborn as sns; sns.set()
#import the csv to panda dataframe
data=pd.read_csv('/content/drive/MyDrive/full.csv')
# set up the stop_words set
stop_words=set(stopwords.words('english'))

/content/drive/MyDrive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<ipython-input-2-7ad83927d3b1>:13: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('/content/drive/MyDrive/full.csv')


In [3]:
#remove duplicates and drop rewards
data=data.drop_duplicates()
data=data.drop(['rewards'],axis=1)
data.duplicated().sum()

0

In [4]:
#get subset the data by only including those from todo, done,in_review, and in_progress
data=data[data['status'].isin(['DONE','TODO','IN_REVIEW','IN_PROGRESS'])]

In [5]:
#checking if there are missing values and we here drop the missing reward rows
data=data.dropna(subset=['reward','description'])
data.isna().sum()

reward            0
id                0
name              1
description       0
dao               0
creat             0
creator           0
audit_log         0
due_date          0
assignees       611
priority          0
status            0
tags           3805
review         6642
done              0
comments         37
subtasks       6302
dtype: int64

In [6]:
# remove the all urls in the description, '#', and newline characters.
def clern_text(text):
    pattern = r'(http\:\/\/|https\:\/\/)?([a-z0-9][a-z0-9\-]*\.)+[a-z][a-z\-]*'
    text = re.sub(pattern, '', text)
    pattern = r'(?:(?!.*?[.]{2})[a-zA-Z0-9](?:[a-zA-Z0-9.+!%-]{1,64}|)|\"[a-zA-Z0-9.+!% -]{1,64}\")@[a-zA-Z0-9][a-zA-Z0-9.-]+(.[a-z]{2,}|.[0-9]{1,})'
    text = re.sub(pattern, '', text)
    text = re.sub(r'(\d)\s+(\d)', r'\1\2', text)
    text=text.replace('#','')
    text=text.strip()
    text=text.lower()
    text=text.replace("\n",'')
    text=text.replace('((.taskid.))','')
    return text.strip()
data.description=data.description.apply(clern_text)

In [7]:
!pip install transformers
from transformers import AutoTokenizer, RobertaModel
model_name = "roberta-base"
model =  RobertaModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.8 MB/s eta 0:00:00


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
encoded_corpus = tokenizer(text=data.description.tolist(),
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=500,
                            return_attention_mask=True)

In [9]:
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

In [10]:
import numpy as np
def filter_long_descriptions(tokenizer, descriptions, max_len):
    indices = []
    lengths = tokenizer(descriptions, padding=False,
                     truncation=False, return_length=True)['length']
    for i in range(len(descriptions)):
        if lengths[i] <= max_len-2:
            indices.append(i)
    return indices
short_descriptions = filter_long_descriptions(tokenizer,
                               data.description.tolist(), 500)
input_ids = np.array(input_ids)[short_descriptions]
attention_mask = np.array(attention_mask)[short_descriptions]
labels = data.reward.to_numpy()[short_descriptions]

Token indices sequence length is longer than the specified maximum sequence length for this model (579 > 512). Running this sequence through the model will result in indexing errors


In [11]:
from sklearn.model_selection import train_test_split
test_size = 0.1
seed = 42
train_inputs, test_inputs, train_labels, test_labels = \
            train_test_split(input_ids, labels, test_size=test_size,
                             random_state=seed)
train_masks, test_masks, _, _ = train_test_split(attention_mask,
                                        labels, test_size=test_size,
                                        random_state=seed)

In [12]:
from sklearn.preprocessing import StandardScaler
price_scaler = StandardScaler()
price_scaler.fit(train_labels.reshape(-1, 1))
train_labels = price_scaler.transform(train_labels.reshape(-1, 1))
original_lables=test_labels
test_labels = price_scaler.transform(test_labels.reshape(-1, 1))

In [13]:
import torch
from torch.utils.data import TensorDataset, DataLoader
batch_size = 32
def create_dataloaders(inputs, masks, labels, batch_size):
    input_tensor = torch.tensor(inputs)
    mask_tensor = torch.tensor(masks)
    labels_tensor = torch.tensor(labels)
    dataset = TensorDataset(input_tensor, mask_tensor,
                            labels_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size,
                            shuffle=True)
    return dataloader
train_dataloader = create_dataloaders(train_inputs, train_masks,
                                      train_labels, batch_size)
test_dataloader = create_dataloaders(test_inputs, test_masks,
                                     test_labels, batch_size)

In [14]:
from transformers import RobertaConfig
configuration = RobertaConfig(hidden_size=240)

In [15]:
import torch.nn as nn
from transformers import AutoTokenizer, RobertaModel
class Regressor(nn.Module):

    def __init__(self, drop_rate=0.2, freeze_camembert=False):

        super(Regressor, self).__init__()
        D_in, D_out = 768, 1

        self.camembert = \
                    RobertaModel(configuration)
        self.regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(D_in, D_out))
    def forward(self, input_ids, attention_masks):

        outputs = self.camembert(input_ids, attention_masks)
        class_label_output = outputs.pooler_output


        outputs = self.regressor(class_label_output)
        return outputs
model = Regressor(drop_rate=0.5)

In [16]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")
model.to(device)

No GPU available, using the CPU instead.


Regressor(
  (camembert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 240, padding_idx=1)
      (position_embeddings): Embedding(512, 240, padding_idx=1)
      (token_type_embeddings): Embedding(2, 240)
      (LayerNorm): LayerNorm((240,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=240, out_features=240, bias=True)
              (key): Linear(in_features=240, out_features=240, bias=True)
              (value): Linear(in_features=240, out_features=240, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=240, out_features=240, bias=True)
              (LayerNorm): LayerNo

In [17]:
from transformers import AdamW
optimizer = AdamW(model.parameters(),
                  lr=0.0001,
                  eps=1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
from transformers import get_linear_schedule_with_warmup
epochs = 5
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                 num_warmup_steps=0, num_training_steps=total_steps)

In [19]:
loss_function = nn.MSELoss()

In [ ]:
from torch.nn.utils.clip_grad import clip_grad_norm
def train(model, optimizer, scheduler, loss_function, epochs,
          train_dataloader, device, clip_value=2):
    for epoch in range(epochs):
        print(epoch)
        print("-----")
        best_loss = 1e10
        model.train()
        for step, batch in enumerate(train_dataloader):
            print(step)
            batch_inputs, batch_masks, batch_labels = \
                               tuple(b.to(device) for b in batch)
            model.zero_grad()
            outputs = model(batch_inputs, batch_masks)
            print(outputs.squeeze().size())
            print(batch_labels.size())
            loss = loss_function(outputs.squeeze().to(torch.float64),
                             batch_labels.squeeze().to(torch.float64))
            print(loss)
            loss.backward()
            clip_grad_norm(model.parameters(), clip_value)
            optimizer.step()
            scheduler.step()

    return model
model = train(model, optimizer, scheduler, loss_function, epochs,
              train_dataloader, device, clip_value=2)

0
-----
0


In [ ]:
len(train_dataloader)

In [ ]:
def evaluate(model, loss_function, test_dataloader, device):
    model.eval()
    test_loss, test_r2 = [], []
    for batch in test_dataloader:
        batch_inputs, batch_masks, batch_labels = \
                                 tuple(b.to(device) for b in batch)
        with torch.no_grad():
            outputs = model(batch_inputs, batch_masks)
        loss = loss_function(outputs, batch_labels)
        test_loss.append(loss.item())
        r2 = r2_score(outputs, batch_labels)
        test_r2.append(r2.item())
    return test_loss, test_r2
def r2_score(outputs, labels):
    labels_mean = torch.mean(labels)
    ss_tot = torch.sum((labels - labels_mean) ** 2)
    ss_res = torch.sum((labels - outputs) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [ ]:
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, _ = \
                                  tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs,
                            batch_masks).view(1,-1).tolist()[0]
    return output

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
mae = mean_absolute_error(y_test, y_pred)
mdae = median_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
mdape = ((pd.Series(y_test) - pd.Series(y_pred))\
         / pd.Series(y_test)).abs().median()
r_squared = r2_score(y_test, y_pred)

In [ ]:
y_pred_scaled = predict(model, test_dataloader, device)


In [ ]:
y_pred_scaled[:10]

In [ ]:
test_labels[:10]

In [ ]:
y_pred = price_scaler.inverse_transform(np.array(y_pred_scaled).reshape(-1, 1))

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(test_labels, y_pred_scaled)


In [ ]:
mse

In [ ]:
original_lables[:10]

In [ ]:
y_pred[:10]

In [ ]:
len(train_dataloader)

In [ ]:
test_labels